## Page Number Pagination

### products/pagination.py

In [ ]:
from rest_framework.pagination import PageNumberPagination


class ProductPagination(PageNumberPagination):
    page_size = 3
    page_query_params = "page" ## El params sera as http://..../product/?page=2
    page_size_query_params = "size" ## dinamimcamente puedes mandar el numero de paginas http://..../product/?size=5 son 5 productos por pagina
    max_page_size = 5 ## maximo de items per page
    last_page_strings = "end" ## Go to last page

### products/serializers.py

In [ ]:
from rest_framework import serliaziers
from .models import Product

class ProductSerializer(serliaziers.ModelSerializers):
    class Meta:
        model = Product 
        fields = ["user", "title", "slug"]

### products/views.py

In [ ]:
from rest_framework import generics
from .pagination import ProductPagination
from .serliaziers import ProductSerializer

class ProductListView(generics ListAPIView):
    queryset = Product.objects.all()
    serliazier_class = ProductSerializer
    pagination_class = ProductPagination

## Limit OffSet Pagination

For a more precise navigation between a big amount of data

- limit: page size
- offset : limit you want to overtaken before selecting the products.

limit = 5
offset = 12

result = it will start from 13 product and take the next 5

### products/pagination.py

In [ ]:
from rest_framework.pagination import PaginationNumberPagination, LimitObjectPagination

...

class ProductLimitObjectPagination(LimitObjectPagination):
    default_limit = 5 
    
    ## http://.../product/?limit=5&offset=5 Incrementa de 5 en 5, inicia en 5 products, se los salta y ofset se vuelve 10
    ## http://.../product/?limit=5&offset=10
    
    max_limit = 7

    ## Even if the limit or size is 10, the max is 7

    limit_query_param = "records"

    ## El parametro se cambia a records
    ## http://.../product/?records=5&offset=10

    offset_query_param = "start"

    ## El parametro offset se cambia a start
    ## http://.../product/?records=5&start=10




### products/views.py

In [ ]:
from rest_framework import generics
from .pagination import ProductLimitObjectPagination
from .serliaziers import ProductSerializer

class ProductListView(generics ListAPIView):
    queryset = Product.objects.all()
    serliazier_class = ProductSerializer
    pagination_class = ProductLimitObjectPagination

## Cursor Pagination

There is no page number only next and previous
The order by default is using the field:
- Created: Date of creation
- Only for news, back page does not work except until you have reach the last page, so only forward to get the next products

### models.py

In [ ]:
class Product(models.Model):
    ...
    created = models.DateTimeField(auto_now_add)= True

### pagination.py

In [ ]:
from rest_framework.pagination import PaginationNumberPagination, LimitObjectPagination, CursorProductPagination

...

class ProductCursorPagination(CursorProductPagination):
    page_size = 4

    ## http://.../product/?cursor=bx00 
    ## does not give the total count of products available
    ## the cursor value is random given by django

    cursor_query_param = "cur"
    ## http://.../product/?cur=bx00 
    ## renames the name

    ordering = "title"
    ## will order by title alphabetic order

    page_size_query_param = 'page_size'
    ## Let the user choose the size of the page

    max_page_size = 100
    ## Max page size

    def get_paginated_response(self, data):
        return Response({
            'links': {
               'next': self.get_next_link(),
               'previous': self.get_previous_link()
            },
            'count': self.page.paginator.count,
            'results': data
        })

    ## Returns links to next page, and back page, plus count and items of the page

### views.py

In [ ]:
from rest_framework import generics
from .pagination import ProductLimitObjectPagination, ProductCursorPagination
from .serliaziers import ProductSerializer

class ProductListView(generics ListAPIView):
    queryset = Product.objects.all()
    serliazier_class = ProductSerializer
    pagination_class = ProductCursorPagination #<----

## Renderers - BrowsableAPI y JSON

http://www.django-rest-framework.org/api-guide/renderers

### config/settings.py

In [ ]:
REST_FRAMEWORK = {
    "DEFAULT_RENDERER_CLASSES" : [
        
        ##SELECT FOR PRODUCTION
        'rest_framework.renderers.JSONRenderer'
        ## OR FOR DEVELOPMENT
        'rest_framework.renderers.BrowsableAPIRenderer"
    ]
}